In [2]:
import pandas as pd
import geopandas as gpd
import itertools
import seaborn as sns

import matplotlib.pyplot as plt
from dbfread import DBF
import numpy as np

In [3]:
# import global TDM functions
import sys
sys.path.insert(0, '../Resources/2-Python/global-functions')
import BigQuery

client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()

In [4]:
hts_hh_23 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.hts_2023_tdm_proc.hh').to_dataframe()
hts_trip_23 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.hts_2023_tdm_proc.trip').to_dataframe()

bq_taz_ustm4 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.geometries.ustm_v4_taz_2025_07_29_geog').to_dataframe()
gdf_taz = gpd.GeoDataFrame(bq_taz_ustm4)

#hts_hh_12 = pd.read_csv(r'source_data_archive/hts-2012/HTS 2012 - db - Household.csv')
#hts_trip_12 = pd.read_csv(r'source_data_archive/hts-2012/HTS 2012 - db - Trip.csv')

In [5]:
gdf_v920_taz = gpd.read_file('data/TAZ/WFv920_TAZ.shp')[['TAZID','CO_TAZID']]

In [6]:
hts_trip_12 = pd.read_csv(r'source-data-archive/hts-trip-tlf-source.csv')#[['weight','PURT7_t','HBSch_lev','PA_AP','Use_for_Dist','depart_per','mode_t','mode_autot','trav_hh','trav_nonhh','trav_tot','tot_trv_wt','WF_Use_Skim','Use_TLF','WF_IIIXXIXX','WF_h_TAZID','WF_o_TAZID','WF_d_TAZID','WF_Time','WF_Dist','WF_FFTime','WF_FFDist','WF_D_Time','WF_D_Dist','WF_D_Gent','WF_D_FFTime','WF_D_FFDist','WF_D_FFGent']]

## Observed TLF Calculation (Previous Method)
The previous method to calculating observed TLF is documented on Model Ace: A:\1 - TDM\2 - Estimate Param\_General Parameters\7 - Update Obs TLF
 0. Data prep (calculate new columns to allow easy processing)
 1. Create Weighted Average Daily Skims
     - using period skims from a recent model run as well as the diurnal pa factors csv file, calculate an average daily distance, general cost, and time skim (voyager script used for this step)
 2. Prepare Truck TLF Data (not explained here)
 3. Prepared Other Purpose TLF Data
     - merge the daily skim data (distance, generalized cost, and time) with the household travel survey trip data
     - create binned table of data
 4. Create Obs TLFs for all trips
     - merge the truck TLF data with the other purpose TLF data
 5. Smooth in Excel
      - the data is then read into excel in undergoes various rounds of smoothing/normalizing/hand adjusting to ensure each TLF curve is smooth, maintains the same average, and looks pretty

## Observed TLF Calculation (New/Proposed Method)
The new/proposed method involves generating PA matrix tables directly from the household travel survey and then letting the model calculate the TLF curves with its current voyager code (which calculates modeled TLF curves). In other words observed "modeled" TLF curves. To do this we will do the following steps:
 0. Data prep (calculate new columns to allow easy processing)
 1. Prepare non-truck non-external PA matrices from the HTS
 2. Prepare external PA matrices from the HTS
      - this involves matching external stations to each IXXI trip
 3. Adjust 4_PA_Distrib_TLF.s to read in new PA observed matrices
 4. run new 4_PA_Distrib_TLF.s to generate observed TLF curves

 5. Prepare truck stuff later



## Data Prep

In [84]:
# merge gdf to get taz level insights
hts_hh_23_merge = hts_hh_23.copy()[['hh_id','hCO_TAZID_USTMv4']]
hts_trip_1 = hts_trip_23.copy().merge(hts_hh_23_merge,how='left',left_on='hh_id',right_on='hh_id')

# household
hts_trip_1 = hts_trip_1.copy().merge(gdf_v920_taz,how='left',left_on='hCO_TAZID_USTMv4',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'hTAZID'})
hts_trip_1['hTAZID'].fillna(-1, inplace=True)

# origin
hts_trip_1 = hts_trip_1.copy().merge(gdf_v920_taz,how='left',left_on='oCO_TAZID_USTMv4',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'oTAZID'})
hts_trip_1['oTAZID'].fillna(-1, inplace=True)

# destination
hts_trip_1 = hts_trip_1.merge(gdf_v920_taz,how='left',left_on='dCO_TAZID_USTMv4',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'dTAZID'})
hts_trip_1['dTAZID'].fillna(-1, inplace=True)

# production
hts_trip_1 = hts_trip_1.merge(gdf_v920_taz,how='left',left_on='pCO_TAZID_USTMv4',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'pTAZID'})
hts_trip_1['pTAZID'].fillna(-1, inplace=True)

# attraction
hts_trip_1 = hts_trip_1.merge(gdf_v920_taz,how='left',left_on='aCO_TAZID_USTMv4',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'aTAZID'})
hts_trip_1['aTAZID'].fillna(-1, inplace=True)

hts_trip_1 = hts_trip_1[['PURP7_t','HBSch_lev','PA_AP','hTAZID','oTAZID','dTAZID','pTAZID','aTAZID','trip_weight_v2']]
hts_trip_1 = hts_trip_1[hts_trip_1['PURP7_t']!='Missing Response']
hts_trip_1

C:\Users\cday\AppData\Local\Temp\ipykernel_27592\2227239038.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hts_trip_1['hTAZID'].fillna(-1, inplace=True)
C:\Users\cday\AppData\Local\Temp\ipykernel_27592\2227239038.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

,PURP7_t,HBSch_lev,PA_AP,hTAZID,oTAZID,dTAZID,pTAZID,aTAZID,trip_weight_v2
37,HBOth,None,PA,1793.0,1793.0,1863.0,1793.0,1863.0,NaN
38,HBOth,None,AP,1793.0,1863.0,1793.0,1793.0,1863.0,NaN
39,HBW,None,PA,1793.0,1793.0,1123.0,1793.0,1123.0,NaN
40,HBW,None,AP,1793.0,1123.0,1793.0,1793.0,1123.0,NaN
41,NHBNW,None,PA,1528.0,1528.0,1520.0,1528.0,1520.0,NaN
...,...,...,...,...,...,...,...,...,...
389084,NHBNW,None,PA,1003.0,1162.0,1158.0,1162.0,1158.0,NaN
389085,NHBNW,None,PA,1003.0,1158.0,1003.0,1158.0,1003.0,NaN
389086,HBShp,None,PA,1003.0,1003.0,1003.0,1003.0,1003.0,NaN
389087,HBOth,None,PA,1003.0,1003.0,1158.0,1003.0,1158.0,NaN


In [85]:
# calculate WF_IIIXXIXX
# Categories:
# - II_Res: hTAZID != -1, pSUBAREA = 1, aSUBAREA = 1, oTAZID != -1, dTAZID != -1
# - II_Vis: hTAZID != -1, pSUBAREA = 1, aSUBAREA = 1, oTAZID != -1, dTAZID != -1
# - IX_AP:  hTAZID != -1  --> pa trip produced outside, attracted inside or ap trip produced inside, attracted outside
# - IX_PA:  hTAZID != -1  --> pa trip produced inside, attracted outside or ap trip produced outside, attracted inside
# - XI_AP:  hTAZID = -1   --> pa trip produced inside, attracted outside or ap trip produced outside, attracted inside
# - XI_PA:  hTAZID = -1   --> pa trip produced outside, attracted inside or ap trip produced inside, attracted outside
hts_trip_2 = hts_trip_1.copy()

h_res = hts_trip_2["hTAZID"] != -1
v_res = hts_trip_2["hTAZID"] == -1
p_in  = hts_trip_2["pTAZID"] != -1
a_in  = hts_trip_2["aTAZID"] != -1
o_in  = hts_trip_2["oTAZID"] != -1
d_in  = hts_trip_2["dTAZID"] != -1

# Conditions (order matters: first match wins)
cond_ii_res = h_res & p_in & a_in & o_in & d_in
cond_ii_vis = v_res & p_in & a_in & o_in & d_in
condi_xx = (~h_res) & (~p_in) &  (~a_in) & (~o_in) & (~d_in)

# IX (resident)
cond_ix_ap = h_res & (
    ((hts_trip_2["PA_AP"] == "PA") & (~p_in) & a_in) |   # PA: produced outside, attracted inside
    ((hts_trip_2["PA_AP"] == "AP") & ( p_in) & ~a_in)    # AP: produced inside, attracted outside
)
cond_ix_pa = h_res & (
    ((hts_trip_2["PA_AP"] == "PA") & ( p_in) & ~a_in) |  # PA: produced inside, attracted outside
    ((hts_trip_2["PA_AP"] == "AP") & (~p_in) &  a_in)    # AP: produced outside, attracted inside
)

# XI (non-resident)
cond_xi_ap = (~h_res) & (
    ((hts_trip_2["PA_AP"] == "PA") & ( p_in) & ~a_in) |  # PA: produced inside, attracted outside
    ((hts_trip_2["PA_AP"] == "AP") & (~p_in) &  a_in)    # AP: produced outside, attracted inside
)
cond_xi_pa = (~h_res) & (
    ((hts_trip_2["PA_AP"] == "PA") & (~p_in) &  a_in) |  # PA: produced outside, attracted inside
    ((hts_trip_2["PA_AP"] == "AP") & ( p_in) & ~a_in)    # AP: produced inside, attracted outside
)


conds = [cond_ii_res, cond_ii_vis, cond_ix_ap, cond_ix_pa, cond_xi_ap, cond_xi_pa, condi_xx]
labels = ["II_Res", "II_Vis", "IX_AP", "IX_PA", "XI_AP", "XI_PA", 'XX']

hts_trip_2["WF_IIIXXIXX"] = np.select(conds, labels, default=np.nan)

In [87]:
hts_trip_3 = hts_trip_2.copy()
hts_trip_3['PURP'] = np.where(
    hts_trip_3['PURP7_t'] == 'HBSch',
    hts_trip_3['HBSch_lev'].map({'primary': 'HBSch_Pr', 'secondary': 'HBSch_Sc'}),
    hts_trip_3['PURP7_t']
)
hts_trip_3

,PURP7_t,HBSch_lev,PA_AP,hTAZID,oTAZID,dTAZID,pTAZID,aTAZID,trip_weight_v2,WF_IIIXXIXX,PURP
37,HBOth,None,PA,1793.0,1793.0,1863.0,1793.0,1863.0,NaN,II_Res,HBOth
38,HBOth,None,AP,1793.0,1863.0,1793.0,1793.0,1863.0,NaN,II_Res,HBOth
39,HBW,None,PA,1793.0,1793.0,1123.0,1793.0,1123.0,NaN,II_Res,HBW
40,HBW,None,AP,1793.0,1123.0,1793.0,1793.0,1123.0,NaN,II_Res,HBW
41,NHBNW,None,PA,1528.0,1528.0,1520.0,1528.0,1520.0,NaN,II_Res,NHBNW
...,...,...,...,...,...,...,...,...,...,...,...
389084,NHBNW,None,PA,1003.0,1162.0,1158.0,1162.0,1158.0,NaN,II_Res,NHBNW
389085,NHBNW,None,PA,1003.0,1158.0,1003.0,1158.0,1003.0,NaN,II_Res,NHBNW
389086,HBShp,None,PA,1003.0,1003.0,1003.0,1003.0,1003.0,NaN,II_Res,HBShp
389087,HBOth,None,PA,1003.0,1003.0,1158.0,1003.0,1158.0,NaN,II_Res,HBOth


## Prepare non-truck non-external PA matrices from the HTS

In [88]:
df_tlf_1 = hts_trip_3.copy()

# filter out XX and nan 
df_tlf_1 = df_tlf_1[df_tlf_1['WF_IIIXXIXX']!='XX']
df_tlf_1 = df_tlf_1[df_tlf_1['WF_IIIXXIXX']!='nan']

# adjust to only PA direction
df_tlf_1['p_taz'] = np.where(df_tlf_1['PA_AP']=='PA', df_tlf_1['pTAZID'], df_tlf_1['aTAZID']).astype(int)
df_tlf_1['a_taz'] = np.where(df_tlf_1['PA_AP']=='PA', df_tlf_1['aTAZID'], df_tlf_1['pTAZID']).astype(int)
df_tlf_1 = df_tlf_1[['WF_IIIXXIXX','PURP','p_taz','a_taz','trip_weight_v2']]
df_tlf_1 = df_tlf_1[df_tlf_1['trip_weight_v2']>0]
df_tlf_1['trip_weight_v2'] = df_tlf_1['trip_weight_v2'].round(2)
df_tlf_1

,WF_IIIXXIXX,PURP,p_taz,a_taz,trip_weight_v2
67,II_Res,NHBNW,1928,3175,468.15
68,II_Res,HBShp,3175,232,679.28
70,II_Res,NHBNW,1162,1158,154.46
71,II_Res,HBShp,1158,1049,169.11
72,II_Res,HBW,1049,1051,110.95
...,...,...,...,...,...
389054,II_Res,HBOth,1086,1262,147.31
389055,II_Res,HBOth,1262,1086,147.31
389072,II_Res,HBShp,532,390,51.42
389073,II_Res,HBShp,532,390,51.42


In [89]:
import pandas as pd
import numpy as np
import openmatrix as omx

# Filter trips for the II_Res and II_Vis groups
df_tlf_ii = df_tlf_1[df_tlf_1['WF_IIIXXIXX'].isin(['II_Res','II_Vis'])].copy()
df_tlf_ii["trip_weight_v2"] = df_tlf_ii["trip_weight_v2"].fillna(0)

# Only take unique TAZs
unique_taz = np.sort(np.union1d(df_tlf_ii["p_taz"].unique(), df_tlf_ii["a_taz"].unique()))
n_taz = len(unique_taz)
taz_map = {orig: new for new, orig in enumerate(unique_taz)}  # map to 0..n_taz-1

# Open Cube-compatible OMX (legacy v1.2)
with omx.open_file("obs_pa_table.omx", "w", version="1.2") as omx_file:
    omx_file.create_mapping("TAZ", list(range(1, n_taz+1)))

    # Initialize total matrix
    total_mat = np.zeros((n_taz, n_taz), dtype=np.float64)

    # First pass: accumulate total matrix only
    for _, g in df_tlf_ii.groupby("PURP"):
        g = g[g["p_taz"].isin(unique_taz) & g["a_taz"].isin(unique_taz)]
        g_agg = g.groupby(["p_taz","a_taz"])["trip_weight_v2"].sum().reset_index()
        rows = g_agg["p_taz"].map(taz_map).values
        cols = g_agg["a_taz"].map(taz_map).values
        vals = g_agg["trip_weight_v2"].values.astype(np.float64)
        mat = np.zeros((n_taz, n_taz), dtype=np.float64)
        mat[rows, cols] = vals
        total_mat += mat

    # Write TOT tab first
    omx_file["TOT"] = total_mat

    # Second pass: write individual purpose tabs
    for purp, g in df_tlf_ii.groupby("PURP"):
        g = g[g["p_taz"].isin(unique_taz) & g["a_taz"].isin(unique_taz)]
        g_agg = g.groupby(["p_taz","a_taz"])["trip_weight_v2"].sum().reset_index()
        rows = g_agg["p_taz"].map(taz_map).values
        cols = g_agg["a_taz"].map(taz_map).values
        vals = g_agg["trip_weight_v2"].values.astype(np.float64)
        mat = np.zeros((n_taz, n_taz), dtype=np.float64)
        mat[rows, cols] = vals
        omx_file[str(purp).upper()] = mat

print("OMX file created successfully with TOT tab first.")


OMX file created successfully with TOT tab first.


In [92]:
df_tlf_ii[df_tlf_ii['PURP']=='HBSch_Sc']['trip_weight_v2'].sum()

130192.1